# Coursera Capstone Project

## Introduction Section

### Project Topic: Where is the best location for our new-open appartment?

#### 1. Project Background Statement

We are looking for a place to start up our new appartment in Texas. Texas is growing very fast recently. A lot of constructions are on going here and there. New appartments, new restaurants can be found anywhere in Texas. As an individual who have lived in Texas for amost five years, I really feel the development of the State. As a third-populated state in the US, living in Texas is very comfortable and relaxed. The traffic is good here as well as the public transporation system.
As the expanding econemy and population of Texas, the demand of new appartment is increasing dramatically. Therefore, we believe that a new appartment business in Texas will be profitable and worth investing.

#### 2. Project Target Statement

In this project, we are going to use Foursquare API to get information and analysis the data we obtained, to make a decision where (which city in Texas) will be the new appartment located.

## Data Section

1. All the city information includling Name, population, household income, crimial rate, etc. (probabily from Webs)
2. popularity of the city - numbers of parks, markets, schools, hospitals. (probabily from Foursquare API)

In [1]:
url1 = 'https://www.tsl.texas.gov/ref/abouttx/popcity32010.html' # city population
url2 = 'https://www.indexmundi.com/facts/united-states/quick-facts/texas/median-household-income/cities#table' # household income data
url3 = 'https://www.indexmundi.com/facts/united-states/quick-facts/texas/average-commute-time/cities#table'# mean travel time to work
url4 = 'http://www.usa.com/rank/texas-state--crime-index--city-rank.htm?hl=&hlst=&wist=&yr=&dis=&sb=ASC&plow=&phigh=&ps=' # city criminal index

In [ ]:
# Install and import all the libraries:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans
!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

import numpy as np

from bs4 import BeautifulSoup

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geopy-1.21.0               |             py_0          58 KB  conda-forge
    openssl-1.1.1g             |       h516909a_0         2.1 MB  conda-forge
    ca-certificates-2020.4.5.1 |       hecc5488_0         146 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    certifi-2020.4.5.1         |   py36h9f0ad1d_0         151 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.50-py_0         conda-forge
    geopy:           1

In [2]:
# create a object to handle the contents of targeting website
# city population
population = requests.get(url1).text
soup_pop = BeautifulSoup(population,'lxml')
# household income
income = requests.get(url2).text
soup_income = BeautifulSoup(income,'lxml')
# average travel time to work
travel_time = requests.get(url3).text
soup_tt = BeautifulSoup(travel_time,'lxml')
# city criminal index 
criminal_index = requests.get(url4).text
soup_ci = BeautifulSoup(criminal_index,'lxml')

NameError: name 'requests' is not defined

Following table is the population of Texas Cities

In [61]:
# find the table in the targeting website and convert them into panda dataframe
# 1. the city population
population_table = soup_pop.find('table')
#print(population_table.prettify())

population_table_rows = population_table.find_all('tr')
population_data = []
for tr in population_table_rows:
    td = tr.find_all('td') 
    row = [i.text.replace('\n','').replace('\t','').replace(' city','') for i in td] # remove \n \t and _city in the text with nothing
    population_data.append(row)

population_data = population_data[1:]
#print (population_data)

df_pop = pd.DataFrame(population_data, columns = ['City Name','Population'])
df_pop = df_pop.drop([0],axis = 0)
df_pop = df_pop.reset_index(drop = True)
df_pop.head(5)
#print(df_pop)

,City Name,Population
0,Houston,"2,099,451"
1,San Antonio,"1,327,407"
2,Dallas,"1,197,816"
3,Austin,"790,390"
4,Fort Worth,"741,206"


In [41]:
print('The size of the table is {}'.format(df_pop.shape))

The size of the table is (1752, 2)


In [66]:
# 2 The household income
income_table = soup_income.findAll('table')[1]
#print(income_table.prettify())

income_table_rows = income_table.find_all('tr')
income_data = []
for tr in income_table_rows:
    td = tr.find_all('td')
    row = [i.text for i in td]
    income_data.append(row)
    
income_data = income_data[1:]
#print(income_data)

df_income = pd.DataFrame(income_data, columns = ['City Name','Household Income Value ($)'])
df_income.head(5)
#print(df_income)

,City Name,Household Income Value ($)
0,Abilene,"47,689"
1,Addison,"73,383"
2,Alamo,"39,275"
3,Alamo Heights,"132,313"
4,Aldine,"38,964"


In [63]:
print('The size of the table is {}'.format(df_income.shape))

The size of the table is (379, 2)


In [ ]:
# 3 Mean time travel to work
tt_table = soup_tt.findAll('table')[1]
#print(tt_table.prettify())

tt_table_rows = income_table.find_all('tr')
tt_data = []
for tr in tt_table_rows:
    td = tr.find_all('td')
    row = [i.text for i in td]
    tt_data.append(row)
    
tt_data = tt_data[1:]
#print(tt_data)

df_tt = pd.DataFrame(tt_data, columns = ['City Name','Mean Travel Time (min)'])
df_tt.head(5)
#print(df_tt)

In [ ]:
# 4 Crime Index
ci_table = soup_ci.find('table')
#print(ci_table.prettify())

ci_table_rows = ci_table.find_all('tr')
ci_data = []
for tr in ci_table_rows:
    td = tr.find_all('td')
    row = [i.text for i in td]
    ci_data.append(row)

ci_data = ci_data[1:]
#print(ci_data)

df_ci = pd.DataFrame(ci_data, columns = ['Rank','Crime Index','City Name/ Population'])
df_ci.head(5)
#print(df_ci)